In [3]:
import numpy as np
import random
import math 
from tqdm import tqdm 

In [4]:
class Environment:
    def __init__(self):
        # encoded racetrack
        self.env = [
                        [0,0,0,0,0,0,0,0],
                        [0,1,1,1,0,0,0,0],
                        [0,1,1,1,1,1,1,1],
                        [0,1,1,0,1,1,1,1],
                        [0,1,1,0,1,0,1,1],
                        [0,1,1,0,1,1,1,1],
                        [0,1,1,1,1,0,1,1],
                        [0,1,1,1,0,0,0,0],
                        [0,1,1,1,0,0,0,0],
                        [0,1,1,1,0,0,0,0],
                        [0,1,1,1,0,0,0,0],
                        [0,1,1,1,0,0,0,0],
                        [0,1,1,1,1,0,0,0],
                        [0,0,1,1,1,0,0,0],
                        [0,0,1,1,1,0,0,0],
                        [0,0,1,1,1,0,0,0],
                        [0,0,1,1,1,0,0,0],
                        [0,0,1,1,1,0,0,0],
                        [0,0,1,1,1,0,0,0],
                        [0,0,1,1,1,0,0,0],
                        [0,0,0,0,0,0,0,0]
                   ]

        self.start = [(19,2),(19,3),(19,4)]
        self.end = [(2,7),(3,7),(4,7),(5,7),(6,7)]
        self.val = [
        (1,1), (2,1), (3,1), (4,1), (5,1), (6,1), (7,1), (8,1), (9,1), (10,1), (11,1), (12,1),
        (1,2), (2,2), (3,2), (4,2), (5,2), (6,2), (7,2), (8,2), (9,2), (10,2), (11,2), (12,2), (13,2), (14,2), (15,2), (16,2), (17,2), (18,2),
        (1,3), (2,3), (6,3), (7,3), (8,3), (9,3), (10,3), (11,3), (12,3), (13,3), (14,3), (15,3), (16,3), (17,3), (18,3),
        (2,4), (3,4), (4,4), (5,4), (6,4), (12,4), (13,4), (14,4), (15,4), (16,4), (17,4), (18,4),
        (2,5), (3,5), (5,5),
        (2,6), (3,6), (4,6), (5,6), (6,6)
        ]
        r,c = self.start[random.randrange(3)]
        self.curr_state = (r,c,0,0)
         

    def reset_state(self):
        x,y = self.start[random.randrange(3)]
        self.curr_state = (x,y,0,0)
        return self.curr_state


    def check_within_track(self,x1,y1,x2,y2):
        x = np.linspace(x2, x1, 11*(x1-x2))
        if (x2 != x1):
          y = y1 + ((y2-y1)/(x2-x1))*(x-x1)
          path = []
          for i in range(len(x)):
            if not((round(x[i]), round(y[i])) in path):
              path.append((round(x[i]), round(y[i])))
          path.reverse() 
          
          for i in path:
              if not(i in self.val) and not(i in self.start) and not(i in self.end): 
                return 1
              elif i in self.end:
                return 2 
          return 0

        else:
          y = np.linspace(y1, y2, (y2-y1)+1)
          path = []
          for i in range(len(y)):
            if not((x1, round(y[i])) in path):
              path.append((x1, round(y[i])))
          path.reverse()
          for i in path:
              if not(i in self.val) and not(i in self.start) and not(i in self.end): 
                return 1
              elif i in self.end:
                return 2
          return 0

    def step(self,state,action):
        x = state[0]
        y = state[1]
        vx_new = state[2] + action[0] 
        vy_new = state[3] + action[1]

        if(vx_new > 5):
            vx_new = 5
        if(vx_new < 0):
            vx_new = 0
        if(vy_new > 5):
            vy_new = 5
        if(vy_new < 0):
            vy_new = 0
        
        if(vx_new == 0 and vy_new == 0):
            vx_new = 0
            vy_new = 1
        
        x_new = x - vx_new
        y_new = y + vy_new
        
        if(x_new < 0 or y_new > 7):
            return (x_new,y_new,vx_new,vy_new), -100, True
        
        if(self.check_within_track(x,y,x_new,y_new) == 1):
            return (x_new,y_new,vx_new,vy_new), -100, True

        if(self.check_within_track(x,y,x_new,y_new) == 2):
            return (x_new,y_new,vx_new,vy_new), 100, True
        
        if(self.check_within_track(x,y,x_new,y_new) == 0):
            return (x_new,y_new,vx_new,vy_new), -1, False
            

In [5]:
class MonteCarlo_Offpolicy:
    def __init__(self):
        self.qsa = dict()
        self.c = dict()
        self.b = dict()
        self.policy = dict()
        self.action = []
        self.reward = []
        self.envi = Environment()
        l = [-1,0,1]
        for i in l:
            for j in l:
                self.action.append((i,j))

        for r in range(21):
            for c in range(8):
                for vx in range(6):
                    for vy in range(6):
                        ac = self.action[random.randrange(0,9)]
                        for act in self.action:
                            if act == ac:
                              self.policy[((r,c,vx,vy), act)] = 1
                            else:
                              self.policy[((r,c,vx,vy), act)] = 0
                            self.qsa[((r,c,vx,vy), act)] = random.randint(-123,100)
                            self.c[((r,c,vx,vy), act)] = 0
                            self.b[((r,c,vx,vy), act)] = 1/9
                            


    def generate_episode(self):
        episode = []
        state = self.envi.reset_state()
        done = False

        while not done:
            l = []
            for act in self.action:
                l.append(self.b[(state,act)])

            action = random.choices(self.action, weights=l, k=1)[0]
            next_state, reward, done = self.envi.step(state,action)
            episode.append([state, action, reward, next_state])
            state = next_state
            if state[0] >= 0 and state[0] <= 20 and state[1] < 8:
              if self.envi.env[state[0]][state[1]] == 0:
                break
        return episode


    
    def algo(self):
        rewards = []
        temp_reward = 0
        itr = 300000
        for i in tqdm(range(itr)):
            episode = self.generate_episode()
            # print(episode)
            g=0
            w=1
            for i in range(len(episode)-1,-1,-1):
                g =  episode[i][2]+0.47*g
                self.c[(episode[i][0],episode[i][1])]+=w                  
                self.qsa[(episode[i][0],episode[i][1])] = self.qsa[(episode[i][0],episode[i][1])] + (w/self.c[(episode[i][0],episode[i][1])]) * (g - self.qsa[(episode[i][0],episode[i][1])])
                # s=episode[i][0]
                # a_max = self.argmax(self.qsa,s)

                q = []
                for action in self.action:
                    q.append(self.qsa[(episode[i][0],action)])
                
                a_star = np.argmax(q)
        
            
                for j in range(0,len(self.action)):
                    if j==a_star:
                        self.b[(episode[i][0],self.action[j])] = 0.8 + 0.2/9
                        self.policy[(episode[i][0],self.action[j])] = 1
                    
                    else:
                        self.b[(episode[i][0],self.action[j])] = 0.2/9
                        self.policy[(episode[i][0],self.action[j])] = 0
                
                if episode[i][1]==self.action[a_star]:
                    w = w/self.b[(episode[i][0],episode[i][1])]
                
                else:
                    break

            if episode[-1][2] == -100:
              temp_reward += len(episode)
            else:
              temp_reward += len(episode) - 1
              rewards.append(temp_reward)
              temp_reward = 0
        self.reward = rewards


In [6]:
policy_obj = MonteCarlo_Offpolicy()
policy_obj.algo() 

100%|██████████| 300000/300000 [15:08<00:00, 330.17it/s]


In [7]:
env = Environment()
episode = []
start = (19,2,0,0)
done = False
prev_state = start
curr_state = start

while not done:
            # po = dict()
            print(curr_state,"\n")
            qsa = []
            for act in policy_obj.action:
                qsa.append(policy_obj.qsa[(prev_state,act)])

            a_star = np.argmax(qsa)
            action = policy_obj.action[a_star]
            curr_state, reward, done = env.step(prev_state,action)
            # valid = self.envi.check_within_track(state[0],state[1],next_state[0],next_state[1])
            episode.append([prev_state, action, reward, curr_state])
            prev_state = curr_state
            # if prev_state[0] >= 0 and prev_state[0] <= 20 and prev_state[1] < 8:
            #   if env.env[prev_state[0]][prev_state[1]] == 0:
            #     break

episode

(19, 2, 0, 0) 

(18, 2, 1, 0) 

(16, 2, 2, 0) 

(13, 2, 3, 0) 

(11, 2, 2, 0) 

(9, 2, 2, 0) 

(7, 3, 2, 1) 

(5, 5, 2, 2) 



[[(19, 2, 0, 0), (1, 0), -1, (18, 2, 1, 0)],
 [(18, 2, 1, 0), (1, -1), -1, (16, 2, 2, 0)],
 [(16, 2, 2, 0), (1, -1), -1, (13, 2, 3, 0)],
 [(13, 2, 3, 0), (-1, -1), -1, (11, 2, 2, 0)],
 [(11, 2, 2, 0), (0, 0), -1, (9, 2, 2, 0)],
 [(9, 2, 2, 0), (0, 1), -1, (7, 3, 2, 1)],
 [(7, 3, 2, 1), (0, 1), -1, (5, 5, 2, 2)],
 [(5, 5, 2, 2), (-1, 0), 100, (4, 7, 1, 2)]]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(np.array(policy_obj.reward))

In [ ]:
env = Environment()
episode = []
start = (19, 3, 0, 0)
done = False
prev_state = start
curr_state = start

while not done:
    print(curr_state,"\n")
    qsa = []
    for action in policy_obj.actions:
        qsa.append(policy_obj.q_table[(prev_state,action)])
    a_star = np.argmax(qsa)
    action = policy_obj.actions[a_star]
    curr_state = env.next_state(prev_state,action)
    episode_end, curr_state = env.check_red(prev_state, curr_state)
    reward = env.reward(curr_state)
    episode.append((prev_state,action,reward,curr_state))
    prev_state = curr_state

episode
    
        

(19, 3, 0, 0) 

(18, 3, 1, 0) 

(16, 3, 2, 0) 

(13, 3, 3, 0) 

(10, 3, 3, 0) 

(7, 3, 3, 0) 

(4, 4, 3, 1) 

(2, 6, 2, 2) 



[((19, 3, 0, 0), (1, 0), -1, (18, 3, 1, 0)),
 ((18, 3, 1, 0), (1, -1), -1, (16, 3, 2, 0)),
 ((16, 3, 2, 0), (1, 0), -1, (13, 3, 3, 0)),
 ((13, 3, 3, 0), (0, -1), -1, (10, 3, 3, 0)),
 ((10, 3, 3, 0), (0, -1), -1, (7, 3, 3, 0)),
 ((7, 3, 3, 0), (0, 1), -1, (4, 4, 3, 1)),
 ((4, 4, 3, 1), (-1, 1), -1, (2, 6, 2, 2)),
 ((2, 6, 2, 2), (-1, 0), 100, (2, 7))]